In [1]:
import cv2 
import time 
import numpy as np
import HandTrackingModule as hand
import mediapipe as mp
import math 
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
width , hight = 648,488


cap = cv2.VideoCapture(0)
cap.set(3,width)
cap.set(4,hight)
detector = hand.handDetector(detectionCon=0.7)



devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)


#volume.GetMute()
#volume.GetMasterVolumeLevel()
volumeRange = volume.GetVolumeRange()
minvol = volumeRange[0]
maxvol = volumeRange[1]
vol = 0
volBar = 400
volPer = 0

while True: 
    success, img = cap.read()
    detector.findHands(img)
    land_mark = detector.findPosition(img,draw= False)
    if len(land_mark) != 0:
    

        x1 ,y1 =  land_mark[4][1],land_mark[4][2] # 4 is thump tip
        x2 ,y2 =  land_mark[8][1],land_mark[8][2] # 8 is index finger tip 
        cx,cy = ((x1+x2)//2,(y1+y2)//2)

        cv2.circle(img, (x1,y1), 7, (0, 0, 255),cv2.FILLED) 
        cv2.circle(img, (x2,y2), 7, (0, 0, 255),cv2.FILLED) 
        cv2.line(img, (x1,y1),(x2,y2),(0, 0, 255),3)
        cv2.circle(img, (cx,cy), 7, (255, 255, 255),cv2.FILLED) 

        length = math.hypot(x2-x1,y2-y1)

        # Hand Range 15 - 200
        # Volume Range -65 - 0

        vol = np.interp(length,[15,190],[minvol,maxvol])
        volBar = np.interp(length, [15,190], [400, 150])
        volPer = np.interp(length, [15,190], [0, 100])
        volume.SetMasterVolumeLevel(vol, None)
        
        
        if length<20 :
            cv2.circle(img, (cx,cy), 7, (0, 0, 0),cv2.FILLED) 

        cv2.rectangle(img, (50, 150), (85, 400), (255,255,255), 3)
        cv2.rectangle(img, (50, int(volBar)), (85, 400), (255,255,255), cv2.FILLED)
        cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,1, (255,255,255), 3)


        


    cv2.imshow("Img", img )
    cv2.waitKey(1)